In [1]:
import os
import sys

file = sys.stdout

In [2]:
model = "dllModel"

In [3]:
build_root = os.path.abspath("Example/")

In [4]:
dll = os.path.join(build_root, model+"_win64.dll")
header = os.path.join(build_root, model+"_ert_shrlib_rtw", model+".h")

In [5]:
assert os.path.exists(dll)
assert os.path.exists(header)

In [6]:
cast_xml_root = os.path.abspath(os.path.join(os.path.curdir, 'castxml'))

In [7]:
import pygccxml

from pygccxml import utils
from pygccxml import declarations
from pygccxml import parser
import ctypes
import os

In [8]:
# Configure the xml generator
xml_generator_config = parser.xml_generator_configuration_t(
    xml_generator_path=r"castxml\bin\castxml.exe",
    xml_generator="castxml",
    compiler_path=r"C:\MATLAB\SupportPackages\R2015b\MW_MinGW_4_9\bin\gcc.exe")

In [9]:
decls = parser.parse([header], xml_generator_config)
ns = decls[0]

INFO Parsing source file "C:\Projects\python_SimulinkDLL\Example\dllModel_ert_shrlib_rtw\dllModel.h" ... 


In [10]:
def ctypes_cppname(CPPNAME):
    """convert a CPPNAME field to a ctypes type."""
    
    # If the CPPNAME ends with a capital _T then it is a Simulink type (but still defined in the field CPPNAME)
    # Return this as is for use when called in other functions.
    if CPPNAME.endswith('_T'):
        return CPPNAME
    # Current specified Simulink types and their corresponding ctype type.
    type_map = {'long long int': 'c_longlong',
                'long long unsigned int': 'c_ulonglong',
                'short unsigned int': 'c_ushort',
                'int': 'c_int',
                'unsigned int': 'c_uint',
                'long int': 'c_long',
                'long unsigned int': 'c_ulong',
                'char': 'c_char',
                'char': 'c_char',
                'signed char': 'c_byte',
                'unsigned char': 'c_ubyte',
                'float': 'c_float',
                'double': 'c_double',
                'short int': 'c_short'}
    # Loop through the simulink definitions and return if found.
    for cppname, ctype in type_map.items():
        if cppname == CPPNAME:
            # Return ctypes.+ for explicitness and to maintain the namespaces.
            return "ctypes."+ctype
    # If we make it this far raise an exception and investigate the CPPNAME.
    # Fix the function or add another type to the type_map.
    raise(Exception("Unhandled data type: "+CPPNAME))

print("# Standard Types")
for typedef in ns.typedefs():
    if hasattr(typedef.decl_type, 'CPPNAME'):
        print("{} = {}".format(typedef.name, ctypes_cppname(typedef.decl_type.CPPNAME)))
        
def var_type(decl_type):
    if hasattr(decl_type, 'CPPNAME'):
        return(ctypes_cppname(decl_type.CPPNAME))
    if pygccxml.declarations.is_array(decl_type):
        if hasattr(decl_type.base, "CPPNAME"):
            ctype = ctypes_cppname(decl_type.base.CPPNAME)
            return "{}*{}".format(ctype.base, decl_type.size)
        elif isinstance(decl_type.base, pygccxml.declarations.cpptypes.declarated_t):
            ctype = decl_type.base.decl_string.split(":")[-1]
            return "{}*{}".format(ctype, decl_type.size)
    if pygccxml.declarations.is_pointer(decl_type):
        return "ctypes.POINTER({})".format(var_type(declarations.remove_pointer(decl_type)))
    if pygccxml.declarations.is_volatile(decl_type):
        return "{}".format(var_type(declarations.remove_volatile(decl_type)))
    if pygccxml.declarations.is_const(decl_type):
        return "{}".format(var_type(declarations.remove_const(decl_type)))
    if isinstance(decl_type, pygccxml.declarations.cpptypes.declarated_t):
        return decl_type.decl_string.split(":")[-1]
    raise Exception(decl_type)
    
def struct_tdef(struct, ns):
    for tdef in ns.typedefs():
        try:
            if tdef.decl_type.declaration == struct:
                return tdef.name
        except:
            continue
    return None

# Standard Types
size_t = ctypes.c_ulonglong
ssize_t = ctypes.c_longlong
intptr_t = ctypes.c_longlong
uintptr_t = ctypes.c_ulonglong
ptrdiff_t = ctypes.c_longlong
wint_t = ctypes.c_ushort
wctype_t = ctypes.c_ushort
errno_t = ctypes.c_int
__time32_t = ctypes.c_long
__time64_t = ctypes.c_longlong
int8_T = ctypes.c_byte
uint8_T = ctypes.c_ubyte
int16_T = ctypes.c_short
uint16_T = ctypes.c_ushort
int32_T = ctypes.c_int
uint32_T = ctypes.c_uint
int64_T = ctypes.c_longlong
uint64_T = ctypes.c_ulonglong
real32_T = ctypes.c_float
real64_T = ctypes.c_double
real_T = ctypes.c_double
time_T = ctypes.c_double
boolean_T = ctypes.c_ubyte
int_T = ctypes.c_int
uint_T = ctypes.c_uint
ulong_T = ctypes.c_ulong
ulonglong_T = ctypes.c_ulonglong
char_T = ctypes.c_char
uchar_T = ctypes.c_ubyte


In [11]:
# Standard Types
size_t = ctypes.c_ulonglong
ssize_t = ctypes.c_longlong
intptr_t = ctypes.c_longlong
uintptr_t = ctypes.c_ulonglong
ptrdiff_t = ctypes.c_longlong
wint_t = ctypes.c_ushort
wctype_t = ctypes.c_ushort
errno_t = ctypes.c_int
__time32_t = ctypes.c_long
__time64_t = ctypes.c_longlong
int8_T = ctypes.c_byte
uint8_T = ctypes.c_ubyte
int16_T = ctypes.c_short
uint16_T = ctypes.c_ushort
int32_T = ctypes.c_int
uint32_T = ctypes.c_uint
int64_T = ctypes.c_longlong
uint64_T = ctypes.c_ulonglong
real32_T = ctypes.c_float
real64_T = ctypes.c_double
real_T = ctypes.c_double
time_T = ctypes.c_double
boolean_T = ctypes.c_ubyte
int_T = ctypes.c_int
uint_T = ctypes.c_uint
ulong_T = ctypes.c_ulong
ulonglong_T = ctypes.c_ulonglong
char_T = ctypes.c_char
uchar_T = ctypes.c_ubyte

In [12]:
def istr(indent=0):
    return " "*(4*indent)

In [13]:
def print_struct(struct, file=sys.stdout, indent=0):
    # Start defining the class.
    print("{}class {}(ctypes.Structure):".format(istr(indent), struct.name), file=file)
    # Define the fields.
    print("{}_fields_ = [".format(istr(indent+1), file=file))
    # For each of the variables inside the struct.
    for var in struct.variables():
        # Loop through and define the field name and type.
        print("{}(\"{}\", {}),".format(istr(indent+2),var.name,var_type(var.decl_type)), file=file)
    # Close our brackets.s
    print("{}]".format(istr(indent+1)), file=file)

    # Get the structure typedef
    tdef = struct_tdef(struct, ns)
    if tdef is not None:
        # If it's none and doesn't match the structure name.
        # Because of how Simulink generates the code and how CastXML reads it structures have different 
        # names than their typedef.
        # 
        # Example: 
        #
        # typedef struct Var_tag {
        #     ...
        # } Var_type;
        # 
        # Would have a structure named Var_tag but a typedef named Var_type.
        
        # Create the typedef.
        if tdef != struct.name:
            print("{}{} = {}".format(istr(indent), tdef, struct.name), file=filetweb)

In [14]:
# Get all of the defined structures.
structs = ns.classes(lambda x: declarations.is_struct(x))
# For each of the structures.
for struct in structs:
    # If the structure has no name, pass.
    if struct.name == "":
        continue
    # If locale is in the structure name, pass (They're not any structures we can access in the DLL)
    if 'locale' in struct.name:
        continue

    print_struct(struct, indent=0)

class tagLC_ID(ctypes.Structure):
    _fields_ = [
        ("wLanguage", ctypes.c_ushort),
        ("wCountry", ctypes.c_ushort),
        ("wCodePage", ctypes.c_ushort),
    ]


NameError: name 'filetweb' is not defined

externs

Only the variables defined as externs are accessible through the dll. Everything else is internal (and not accessible.)

To grab a reference to the variable we access it with "TYPE.in_dll(dll, VARIABLE_NAME)"

In [15]:
externs = ns.variables(lambda v: v.type_qualifiers.has_extern)
for extern in externs:
#    extern_type = extern.decl_type.declaration.name
    print("{} = {}.in_dll(dll, '{}')".format(extern.name, var_type(extern.decl_type), extern.name))

dllModel_P = P_dllModel_T.in_dll(dll, 'dllModel_P')
dllModel_B = B_dllModel_T.in_dll(dll, 'dllModel_B')
dllModel_U = ExtU_dllModel_T.in_dll(dll, 'dllModel_U')
dllModel_Y = ExtY_dllModel_T.in_dll(dll, 'dllModel_Y')
SignalIn = real32_T.in_dll(dll, 'SignalIn')
SimulationSignal2 = real_T.in_dll(dll, 'SimulationSignal2')
SignalOut = real32_T.in_dll(dll, 'SignalOut')
K = real32_T.in_dll(dll, 'K')
dllModel_M = ctypes.POINTER(tag_RTM_dllModel_T).in_dll(dll, 'dllModel_M')


In [16]:
# Development tool.
def vardump(var):
    fields = dir(var)
    for field in fields:
        if field.startswith("_"):
            continue
        try:
            f = getattr(var, field)
        except:
            continue
        try:
            print("{}(): {}".format(field, f()))
        except:
            try:
                print("{}: {}<{}>".format(field, type(f), f))
            except:
                pass